In [1]:
import os
import sys
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
file_masterdata  = r'C:\Users\vanandp\Documents\WORK\DataLake\private\nse_stock_analysis\data\output\nifty_masterdata.xlsx'
output_file = r'C:\Users\vanandp\Documents\WORK\DataLake\private\nse_stock_analysis\data\output\52W_insight.xlsx'

In [3]:
df = pd.read_excel(file_masterdata)
pd.options.display.max_columns = None

In [4]:
df.columns

Index(['Company Name', 'Industry', 'Symbol', 'Series', 'ISIN Code', 'TAGS',
       'TAG_SCORE', 'ALGO_SCORE', 'adhocMargin', 'applicableMargin',
       'averagePrice', 'basePrice', 'bcEndDate', 'bcStartDate', 'buyPrice1',
       'buyPrice2', 'buyPrice3', 'buyPrice4', 'buyPrice5', 'buyQuantity1',
       'buyQuantity2', 'buyQuantity3', 'buyQuantity4', 'buyQuantity5',
       'change', 'closePrice', 'cm_adj_high_dt', 'cm_adj_low_dt', 'cm_ffm',
       'companyName', 'css_status_desc', 'dayHigh', 'dayLow',
       'deliveryQuantity', 'deliveryToTradedQuantity', 'exDate',
       'extremeLossMargin', 'faceValue', 'high52', 'indexVar', 'isExDateFlag',
       'isinCode', 'lastPrice', 'low52', 'marketType', 'ndEndDate',
       'ndStartDate', 'open', 'pChange', 'previousClose', 'priceBand',
       'pricebandlower', 'pricebandupper', 'purpose', 'quantityTraded',
       'recordDate', 'secDate', 'securityVar', 'sellPrice1', 'sellPrice2',
       'sellPrice3', 'sellPrice4', 'sellPrice5', 'sellQuantity1'

In [5]:
df.shape[0]

501

In [6]:
columns_order = [
    'Series',  'Industry', 'Company Name', 'Symbol', '52wHigh', '52wLow', '52wDiff', '52wMid',  
    'avgPrice', 'Mid%', 'avgP-52wMid', 'Low%', 'avgP-52wLow',
    'dayHigh', 'dayLow', 'dayDiff', 'change', 'pChange', 'open', 'lastPrice', 'prevClose', 'closePrice', 'basePrice', 
    '52wHighDate', '52wLowDate', '52wDateDiff', 'ALGO_SCORE',  'TAGS', 'TAG_SCORE', 'adhocMargin', 
    'applicableMargin', 'bcEndDate', 'bcStartDate', 'buyPrice1',
    'buyPrice2', 'buyPrice3', 'buyPrice4', 'buyPrice5', 'buyQuantity1', 'buyQuantity2', 'buyQuantity3', 'buyQuantity4', 
    'buyQuantity5', 'cm_ffm', 'companyName', 'css_status_desc', 'deliveryQuantity', 'deliveryToTradedQuantity', 'exDate', 
    'extremeLossMargin', 'faceValue', 'indexVar', 'isExDateFlag', 'isinCode', 'marketType', 'ndEndDate', 'ndStartDate', 
    'priceBand', 'pricebandlower', 'pricebandupper', 'purpose', 'quantityTraded', 'recordDate', 'secDate', 'securityVar', 
    'sellPrice1', 'sellPrice2', 'sellPrice3', 'sellPrice4', 'sellPrice5', 'sellQuantity1', 'sellQuantity2', 'sellQuantity3', 
    'sellQuantity4', 'sellQuantity5', 'series', 'surv_indicator', 'symbol', 'totalBuyQuantity', 'totalSellQuantity', 
    'totalTradedValue', 'totalTradedVolume', 'varMargin']
    
    
rename_columns = {'previousClose': 'prevClose',
                 'averagePrice': 'avgPrice',
                  'cm_adj_high_dt': '52wHighDate',
                  'cm_adj_low_dt': '52wLowDate',
                  'high52': '52wHigh',
                  'low52': '52wLow'
                 }


def modify_columns(df):
    df.rename(index=str, columns=rename_columns, inplace=True)  # rename
    return df

def reorder_columns(df):
    all_col = list(df.columns)
    diff_col = set(all_col) - set(columns_order) 
    diff_col = list(diff_col)
    diff_col.sort()
    print(diff_col)
    df = df[columns_order + diff_col]
    return df

def convert_float(x):
    try:
        return float(x)
    except:
        return float(0.0)

In [7]:
import os
if os.path.exists(output_file):
    os.remove(output_file)

def cost_per_range(cost):
    if 0 <= cost < 50:
        return 0
    elif 50 <= cost < 100:
        return 50
    elif 100 <= cost < 200:
        return 100
    elif 200 <= cost < 400:
        return 200
    elif 400 <= cost < 600:
        return 400
    elif 600 <= cost < 800:
        return 600
    elif 800 <= cost < 1000:
        return 800
    elif 1000 <= cost:
        return 1000


    
def format_nifty(writer_obj, sheet_name, df):
    num_rows = df.shape[0] + 1
    num_cols = df.shape[1] + 1
    
    print('Formating the columns - {}'.format(sheet_name))
    writer = writer_obj
    workbook = writer.book
    writer.book.strings_to_formulas = False
    cell_fmt = workbook.add_format({'align': 'right', 'bold': False, 'border': True})
    test_fmt = workbook.add_format({'bold': True, 'align': 'bottom', 'bg_color': '#D7E4BC'})
    title_fmt = workbook.add_format({'bold': True, 'bg_color': '#59ddff',  'align': 'center',
                                     'valign': 'vcenter', 'fg_color': '#D7E4BC', 'border': 1, 'font_color': 'black'})
    worksheet = writer.sheets[sheet_name]
    # Hide all rows without data.
    worksheet.set_default_row(hide_unused_rows=True)
    worksheet.freeze_panes(1, 0)
    worksheet.set_zoom(80)
    # green - high - red - low 
    high_color_codes = { 100: '#00cc00' , 80: '#00ff00' , 60: '#66ff99', 40: '#ccffcc' , 20: '#e6ffe6', 0: '#ffffff' }
    low_color_codes = { 100: '#ff0000' , 80: '#ff794d', 60: '#ff4d4d', 40: '#ff8080', 20: '#ffcccc', 0: '#ffffff'}
    cost_per_codes = {1000: '#009933', 800: '#00b33c', 600: '#00e64d', 400: '#1aff66', 200: '#4dff88', 100: '#80ffaa',
                       50: '#b3ffcc', 0: '#ffffff'}
    # column
    column_width = {'A': 5, 'AA': 8, 'B': 27, 'AB': 8, 'C': 45, 'AC': 8, 'D': 14, 'AD': 8, 'E': 8, 'AE': 8, 'F': 8, 'AF': 8, 
                    'G': 8, 'AG': 8, 'H': 8, 'AH': 8, 'I': 8, 'AI': 8, 'J': 8, 'AJ': 8, 'K': 8, 'AK': 8, 'L': 8, 'AL': 8, 
                    'M': 8, 'AM': 8, 'N': 8, 'AN': 8, 'O': 8, 'AO': 8, 'P': 8, 'AP': 8, 'Q': 18, 'AQ': 8, 'R': 18, 'AR': 8, 
                    'S': 8, 'AS': 8, 'T': 8, 'AT': 8, 'U': 12, 'AU': 8, 'V': 12, 'AV': 8, 'W': 11, 'AW': 8, 'X': 8, 'AX': 8, 
                    'Y': 8, 'AY': 8, 'Z': 8, 'AZ': 8, 'BA': 8, 'BB': 8, 'BC': 8, 'BD': 8, 'BE': 8, 'BF': 8, 'BG': 8, 
                    'BH': 8, 'BI': 8, 'BJ': 8, 'BK': 8, 'BL': 8, 'BM': 8, 'BN': 8, 'BO': 8, 'BP': 8, 'BQ': 8, 'BR': 8, 
                    'BS': 8, 'BT': 8, 'BU': 8, 'BV': 8, 'BW': 8, 'BX': 8, 'BY': 8, 'BZ': 8}
    
    worksheet.set_row(0, 18, title_fmt)
    worksheet.autofilter('A1:BZ{}'.format(num_rows))
    for idx, key in enumerate(column_width.keys()):
        worksheet.conditional_format("{}1".format(key), {'type': 'no_errors', 'format': title_fmt})
    for key in column_width.keys():
        worksheet.set_column("{}:{}".format(key, key), column_width[key], cell_fmt)

    for i in range (0, len(df)):
        diff52w = df['52wDiff'].iloc[i].round(2)
        lowPer = df['Low%'].iloc[i]
        diffLow = df['avgP-52wLow'].iloc[i]
        lowPrice = df['52wLow'].iloc[i].round(2)
        midPrice = df['52wMid'].iloc[i].round(2)
        avgPrice = df['avgPrice'].iloc[i].round(2)
        diffPrice = round(avgPrice - midPrice, 2)
        perChange = round((diffPrice / midPrice) * 100, 2)
        # percent
        if  80 <= abs(perChange):
            color_code = 100
        elif 60 <= abs(perChange) < 80:
            color_code = 80
        elif 40 <= abs(perChange) < 60:
            color_code = 60
        elif 20 <= abs(perChange) < 40:
            color_code = 40
        elif  abs(perChange) < 20:
            color_code = 20
        else:
            color_code = 0
        # color_code
        if perChange > 0:
            color_code = high_color_codes[color_code]
        else:
            color_code = low_color_codes[color_code]
        perChange = diffPrice / midPrice
        cell_fmt0 = workbook.add_format({'num_format': '#,###', 'border': 1})
        cell_fmt1 = workbook.add_format({'num_format': '#,###', 'fg_color': color_code.strip(), 'border': 1})
        cell_fmt2 = workbook.add_format({'num_format': '0.00%', 'fg_color': color_code.strip(), 'border': 1})
        cell_fmt3 = workbook.add_format({'num_format': '#,###', 'fg_color': cost_per_codes[cost_per_range(diff52w)], 'border': 1})
        cell_fmt4 = workbook.add_format({'num_format': '0.00%', 'fg_color': cost_per_codes[cost_per_range(lowPer*100)], 'border': 1})

        # cost per range
        worksheet.write('G{}'.format(i+2), diff52w, cell_fmt3)
        # high / low
        worksheet.write('H{}'.format(i+2), midPrice, cell_fmt1)
        worksheet.write('I{}'.format(i+2), avgPrice, cell_fmt1)
        worksheet.write('J{}'.format(i+2), perChange, cell_fmt2)  
        # cost per range
        worksheet.write('M{}'.format(i+2), diffLow, cell_fmt0)
        worksheet.write('K{}'.format(i+2), diffPrice, cell_fmt0) 
        worksheet.write('L{}'.format(i+2), lowPer, cell_fmt4)  
    worksheet.conditional_format('Q2:Q{}'.format(num_rows), {'type': 'data_bar', 'data_bar_2010': True})
    worksheet.conditional_format('R2:R{}'.format(num_rows), {'type': 'data_bar', 'data_bar_2010': True})
    worksheet.conditional_format('T2:T{}'.format(num_rows), {'type': '3_color_scale'})
    worksheet.conditional_format('K2:K{}'.format(num_rows), {'type': '3_color_scale'})
    worksheet.conditional_format('M2:M{}'.format(num_rows), {'type': '3_color_scale'})
    worksheet.conditional_format('P2:P{}'.format(num_rows), {'type': '3_color_scale'})

    


# Main Call
writer = pd.ExcelWriter(output_file, engine='xlsxwriter', datetime_format='mmm dd yyyy',
                        date_format='mmmm dd yyyy', options={'strings_to_urls': False, 'strings_to_formulas': False})

df = modify_columns(df)
df['dayDiff'] = df['dayHigh'] - df['dayLow']
df['52wDiff'] = df['52wHigh'] - df['52wLow']
df['52wDateDiff'] = df['52wLowDate'] - df['52wHighDate']
df['52wMid'] = (df['52wHigh'] + df['52wLow']) / 2
df['avgP-52wMid'] = df['avgPrice'] - df['52wMid']
df['Mid%'] = ((df['avgPrice'] - df['52wMid']) /  df['52wMid']) * 100
df['Low%'] = (df['avgPrice'] - df['52wLow']) /  df['52wLow']
df['avgP-52wLow'] = df['avgPrice'] - df['52wLow']
df['change'] = df['change'].apply(convert_float)

df = reorder_columns(df)
df.sort_values(by=['Mid%'], ascending=False, inplace=True)
df.to_excel(writer, sheet_name='Nifty', index=False)
format_nifty(writer, sheet_name='Nifty', df=df)


['ISIN Code']
Formating the columns - Nifty


In [145]:
# green - high
high_color_codes = { 100: '#00cc00' , 80: ' #33ff33' , 60: '#99ff99', 40: ' #ccffcc' , 20: '#b3ffb3', 0: '#ffffff' }
low_color_codes = { 100: '#ff3300' , 80: '#ff1a1a', 60: '#ff4d4d', 40: ' #ff8080', 20: '#ffcccc', 0: '#ffffff'}

for i in range (0, len(df)):
    
    midPrice = df['52wMid'].iloc[i].round(2)
    avgPrice = df['avgPrice'].iloc[i].round(2)
    diffPrice = round(avgPrice - midPrice, 2)
    perChange = round((diffPrice / midPrice) * 100, 2)
    
    if  80 <= abs(perChange):
        color_code = 100
    elif 60 <= abs(perChange) < 80:
        color_code = 80
    elif 40 <= abs(perChange) < 60:
        color_code = 60
    elif 20 <= abs(perChange) < 40:
        color_code = 40
    elif  abs(perChange) < 20:
        color_code = 20
    else:
        color_code = 0
    if perChange > 0:
        color_code = high_color_codes[color_code]
    else:
        color_code = low_color_codes[color_code]
        
    print(midPrice, avgPrice, diffPrice, perChange, color_code)
#     if df['Status'].ix[i] == "y":
#         worksheet.write(i+1, 0, df['Name'].ix[i], format1)
#     elif df['Status'].ix[i] == "n":
#         worksheet.write(i+1, 0, df['Name'].ix[i], format2)
#     elif df['Status'].ix[i] == "yy":
#         worksheet.write(i+1, 0, df['Name'].ix[i], format3)


2502.0 4240.94 1738.94 69.5  #33ff33
683.6 1102.73 419.13 61.31  #33ff33
481.4 758.23 276.83 57.51 #99ff99
481.4 758.23 276.83 57.51 #99ff99
397.65 577.03 179.38 45.11 #99ff99
912.18 1318.66 406.48 44.56 #99ff99
181.27 256.35 75.08 41.42 #99ff99
209.52 296.01 86.49 41.28 #99ff99
1304.4 1842.81 538.41 41.28 #99ff99
2037.0 2837.3 800.3 39.29  #ccffcc
144.6 199.82 55.22 38.19  #ccffcc
144.6 199.82 55.22 38.19  #ccffcc
1096.01 1514.09 418.08 38.15  #ccffcc
265.75 363.14 97.39 36.65  #ccffcc
1175.38 1594.89 419.51 35.69  #ccffcc
770.12 1026.87 256.75 33.34  #ccffcc
25407.55 33712.91 8305.36 32.69  #ccffcc
910.38 1193.03 282.65 31.05  #ccffcc
2209.6 2859.54 649.94 29.41  #ccffcc
1078.22 1391.1 312.88 29.02  #ccffcc
19730.05 25444.74 5714.69 28.96  #ccffcc
366.55 467.64 101.09 27.58  #ccffcc
2384.5 3038.79 654.29 27.44  #ccffcc
197.6 251.57 53.97 27.31  #ccffcc
319.27 404.78 85.51 26.78  #ccffcc
920.6 1166.29 245.69 26.69  #ccffcc
1097.62 1388.52 290.9 26.5  #ccffcc
17777.92 22420.32 4642.4 2

174.15 143.94 -30.21 -17.35 #ffcccc
241.7 199.74 -41.96 -17.36 #ffcccc
447.9 369.39 -78.51 -17.53 #ffcccc
324.73 267.61 -57.12 -17.59 #ffcccc
258.75 212.21 -46.54 -17.99 #ffcccc
371.48 303.93 -67.55 -18.18 #ffcccc
1355.5 1108.57 -246.93 -18.22 #ffcccc
144.32 117.98 -26.34 -18.25 #ffcccc
128.1 104.67 -23.43 -18.29 #ffcccc
771.32 628.32 -143.0 -18.54 #ffcccc
72.95 59.19 -13.76 -18.86 #ffcccc
236.5 191.7 -44.8 -18.94 #ffcccc
134.95 109.38 -25.57 -18.95 #ffcccc
98.28 79.5 -18.78 -19.11 #ffcccc
234.98 190.07 -44.91 -19.11 #ffcccc
84.18 68.03 -16.15 -19.19 #ffcccc
363.0 293.28 -69.72 -19.21 #ffcccc
145.05 117.19 -27.86 -19.21 #ffcccc
340.12 274.79 -65.33 -19.21 #ffcccc
376.0 303.42 -72.58 -19.3 #ffcccc
310.0 250.12 -59.88 -19.32 #ffcccc
349.9 281.79 -68.11 -19.47 #ffcccc
971.58 782.2 -189.38 -19.49 #ffcccc
94.28 75.67 -18.61 -19.74 #ffcccc
136.5 109.49 -27.01 -19.79 #ffcccc
98.8 78.98 -19.82 -20.06  #ff8080
325.83 260.46 -65.37 -20.06  #ff8080
278.75 222.73 -56.02 -20.1  #ff8080
253.2 201.65

In [96]:
abs(69.5) > 60

True

In [56]:
df.head(5)

,Series,Industry,Company Name,Symbol,52wHigh,52wLow,52wDiff,52wMid,avgPrice,avgP-52wMid,open,lastPrice,prevClose,closePrice,basePrice,dayHigh,dayLow,dayDiff,change,pChange,52wHighDate,52wLowDate,52wDateDiff,ALGO_SCORE,ISIN Code,TAGS,TAG_SCORE,adhocMargin,applicableMargin,bcEndDate,bcStartDate,buyPrice1,buyPrice2,buyPrice3,buyPrice4,buyPrice5,buyQuantity1,buyQuantity2,buyQuantity3,buyQuantity4,buyQuantity5,cm_ffm,companyName,css_status_desc,deliveryQuantity,deliveryToTradedQuantity,exDate,extremeLossMargin,faceValue,indexVar,isExDateFlag,isinCode,marketType,ndEndDate,ndStartDate,priceBand,pricebandlower,pricebandupper,purpose,quantityTraded,recordDate,secDate,securityVar,sellPrice1,sellPrice2,sellPrice3,sellPrice4,sellPrice5,sellQuantity1,sellQuantity2,sellQuantity3,sellQuantity4,sellQuantity5,series,surv_indicator,symbol,totalBuyQuantity,totalSellQuantity,totalTradedValue,totalTradedVolume,varMargin
0,EQ,ENERGY,Jaiprakash Power Ventures Ltd.,JPPOWER,10.20,2.40,7.8,6.3,2.94,-3.36,2.85,2.95,2.85,2.90,2.85,3.00,2.85,0.15,0.10,3.51,2018-01-01,2018-07-24,204 days,5.5,INE351F01018,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,25.31,15-SEP-17,08-SEP-17,2.90,-,-,-,-,123295,-,-,-,-,1095.47,Jaiprakash Power Ventures Limited,Listed,1242475,36.98,06-SEP-17,6.06,10,-,0,INE351F01018,N,-,-,20,2.30,3.40,ANNUAL GENERAL MEETING,3359651,-,17AUG2018,19.25,-,-,-,-,-,-,-,-,-,-,EQ,-,JPPOWER,123295,-,98.77,3359651,19.25
1,EQ,CONSTRUCTION,Unitech Ltd.,UNITECH,12.25,3.95,8.3,8.1,4.25,-3.85,4.10,4.25,4.05,4.30,4.05,4.35,4.10,0.25,0.20,4.94,2018-01-02,2018-07-17,196 days,5.5,INE694A01020,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,15.20,28-SEP-17,23-SEP-17,4.30,-,-,-,-,5807620,-,-,-,-,956.26,Unitech Limited,Listed,9031033,43.08,21-SEP-17,5.00,2,-,0,INE694A01020,N,-,-,20,3.25,4.85,ANNUAL GENERAL MEETING,20965259,-,17AUG2018,10.20,-,-,-,-,-,-,-,-,-,-,EQ,-,UNITECH,5807620,-,891.02,20965259,10.20
2,EQ,ENERGY,RattanIndia Power Ltd.,RTNPOWER,9.50,3.10,6.4,6.3,4.14,-2.16,4.10,4.10,4.05,4.10,4.05,4.25,4.10,0.15,0.05,1.23,2017-11-21,2018-07-06,227 days,5.5,INE399K01017,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,17.44,19-SEP-17,13-SEP-17,-,-,-,-,-,-,-,-,-,-,556.92,RattanIndia Power Limited,Listed,995035,56.54,11-SEP-17,5.00,10,-,0,INE399K01017,N,-,-,20,3.25,4.85,ANNUAL GENERAL MEETING,1759737,-,17AUG2018,12.44,-,-,-,-,-,-,-,-,-,-,EQ,-,RTNPOWER,-,-,72.85,1759737,12.44
3,EQ,CONSUMER GOODS,Bajaj Hindusthan Sugar Ltd.,BAJAJHIND,18.80,5.60,13.2,12.2,6.56,-5.64,6.55,6.50,6.50,6.55,6.50,6.65,6.50,0.15,0.00,0.00,2018-01-02,2018-07-20,199 days,5.5,INE306A01021,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,14.81,15-SEP-17,09-SEP-17,-,-,-,-,-,-,-,-,-,-,601.41,Bajaj Hindusthan Sugar Limited,Listed,404411,60.69,07-SEP-17,5.00,1,-,0,INE306A01021,N,-,-,20,5.20,7.80,ANNUAL GENERAL MEETING,666333,-,17AUG2018,9.81,6.55,-,-,-,-,35586,-,-,-,-,EQ,-,BAJAJHIND,-,35586,43.71,666333,9.81
4,EQ,INDUSTRIAL MANUFACTURING,Suzlon Energy Ltd.,SUZLON,17.75,6.85,10.9,12.3,7.24,-5.06,7.25,7.20,7.20,7.25,7.20,7.30,7.20,0.10,0.00,0.00,2017-08-16,2018-07-19,337 days,12.5,INE040H01021,"nifty200, nifty500, niftylargemidcap250, nifty...",5,-,17.09,27-JUL-18,21-JUL-18,-,-,-,-,-,-,-,-,-,-,2391.24,Suzlon Energy Limited,Listed,4274193,38.17,19-JUL-18,5.00,2,-,0,INE040H01021,N,-,-,No Band,6.50,7.90,ANNUAL GENERAL MEETING,11198311,-,17AUG2018,12.09,7.25,-,-,-,-,259416,-,-,-,-,EQ,-,SUZLON,-,259416,810.76,11198311,12.09


In [57]:
# 52 Week Insights
import xlsxwriter
import xlrd
import openpyxl
from datetime import datetime as dt
import calendar


month_sell = df["52wHighDate"].dt.month.apply(lambda x: calendar.month_name[x]).value_counts().to_dict()
month_buy = df["52wLowDate"].dt.month.apply(lambda x: calendar.month_name[x]).value_counts().to_dict()

day_sell = df["52wHighDate"].dt.day.value_counts().to_dict()
day_buy = df["52wLowDate"].dt.day.value_counts().to_dict()

year_sell = df["52wHighDate"].dt.year.value_counts().to_dict()
year_buy = df["52wLowDate"].dt.year.value_counts().to_dict()

# right days to buy
week_day_sell= df["52wHighDate"].dt.weekday_name.value_counts().to_dict()
week_day_buy = df["52wLowDate"].dt.weekday_name.value_counts().to_dict()

quarter_sell = df["52wHighDate"].dt.quarter.value_counts().to_dict()
quarter_buy = df["52wLowDate"].dt.quarter.value_counts().to_dict()

# Long term investment week number  
week_sell =df["52wHighDate"].dt.week.value_counts().to_dict()
week_buy =df["52wLowDate"].dt.week.value_counts().to_dict()

today = dt.now()
quarter = int(today.strftime('%m')) // 3 + 1
month = today.strftime('%B')
day = int(today.strftime('%d'))
week_num = int(today.strftime('%V'))
weekday = today.strftime('%A')

workbook = xlsxwriter.Workbook(output_file)
worksheet = workbook.add_worksheet('52W Stats')


# Set tab colors
worksheet.set_tab_color('#FF9900')  # Orange

workbook.set_properties({
    'title':    '52Week Stats',
    'subject':  'With document properties',
    'author':   'John McNamara',
    'manager':  'Dr. Vijay',
    'company':  'Awesome Work Pays',
    'category': 'Example spreadsheets',
    'keywords': 'Sample, Example, Properties',
    'comments': 'Created with Python and XlsxWriter',
    'status':   'Quo',
})

diag_format = workbook.add_format({'diag_type': 3,'diag_border': 1,'diag_color': 'red',})
buy_format = workbook.add_format({'bold': True, 'font_color': 'green', 'align': 'center'})
sell_format = workbook.add_format({'bold': True, 'font_color': 'red', 'align': 'center'})
bold_format = workbook.add_format({'bold': True})
title_format = workbook.add_format({'bold': True, 'align': 'center', 'bg_color': 'yellow', 'border': True})
sell1_format = workbook.add_format({'bg_color': '#FF5733', 'border': True})
sell2_format = workbook.add_format({'bg_color': '#F1948A'})
sell3_format = workbook.add_format({'bg_color': '#FADBD8'})
buy1_format = workbook.add_format({'bg_color': '#10CF07', 'border': True})
buy2_format = workbook.add_format({'bg_color': '#2ECC71'})
buy3_format = workbook.add_format({'bg_color': '#ABEBC6'})

worksheet.set_column('A:A')
worksheet.write('A1', "Month", title_format)
# worksheet.add_table('A2:D14', {'header_row': False, 'style': 'Table Style Light 11'})
worksheet.write('A2', "Buy", buy_format)
worksheet.write('C2', "Sell", sell_format)
row = 2
col = 0
# MONTH BUY
for item, cost in (list(month_buy.items())):
    if item == month:
        worksheet.write(row, col, item, buy1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
row = 2
col = 2
# MONTH SELL
for item, cost in (list(month_sell.items())):
    if item == month:
        worksheet.write(row, col, item, buy1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
    

worksheet.write('A16', "Weekday", title_format)
# worksheet.add_table('A17:D22', {'header_row': False, 'style': 'Table Style Light 11'})
worksheet.write('A17', "Buy", buy_format)
worksheet.write('C17', "Sell", sell_format)
row = 17
col = 0
# WEEKDAY BUY
for item, cost in (list(week_day_buy.items())):
    if item == weekday:
        worksheet.write(row, col, item, buy1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
row = 17
col = 2
# WEEKDAY SELL
for item, cost in (list(week_day_sell.items())):
    if item == weekday:
        worksheet.write(row, col, item, sell1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1

worksheet.write('A24', "Quarter", title_format)
worksheet.add_table('A25:D29', {'header_row': False, 'style': 'Table Style Light 11'})
worksheet.write('A25', "Buy", buy_format)
worksheet.write('C25', "Sell", sell_format)
row = 25
col = 0
# Quarter BUY
for item, cost in (list(quarter_buy.items())):
    if item == quarter:
        worksheet.write(row, col, item, buy1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
row = 25
col = 2
# Quarter SELL
for item, cost in (list(quarter_sell.items())):
    if item == quarter:
        worksheet.write(row, col, item, sell1_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1


worksheet.write('F1', "Day", title_format)
# worksheet.add_table('F3:I33', {'header_row': False, 'style': 'Table Style Light 9'})
worksheet.write('F2', "Buy", buy_format)
worksheet.write('H2', "Sell", sell_format)
row = 2
col = 5
# Day BUY
for item, cost in (list(day_buy.items())):
    if int(item) == day:
        worksheet.write(row, col, item, buy1_format)
    elif int(item) == day+1:
        worksheet.write(row, col, item, buy2_format)
    elif int(item) == day-1:
        worksheet.write(row, col, item, buy3_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
row = 2
col = 7
# Day SELL
for item, cost in (list(day_sell.items())):
    if int(item) == day:
        worksheet.write(row, col, item, sell1_format)
    elif int(item) == day+1:
        worksheet.write(row, col, item, sell2_format)
    elif int(item) == day-1:
        worksheet.write(row, col, item, sell3_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1

worksheet.write('K1', "Week", title_format)
# worksheet.add_table('K3:N50', {'header_row': False, 'style': 'Table Style Light 13'})
worksheet.write('K2', "Buy", buy_format)
worksheet.write('M2', "Sell", sell_format)
row = 2
col = 10
# Week BUY
for item, cost in (list(week_buy.items())):
    if int(item) == week_num:
        worksheet.write(row, col, item, buy1_format)
    elif int(item) == week_num+1:
        worksheet.write(row, col, item, buy2_format)
    elif int(item) == week_num-1:
        worksheet.write(row, col, item, buy3_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1
row = 2
col = 12
# Week SELL
for item, cost in (list(week_sell.items())):
    if int(item) == week_num:
        worksheet.write(row, col, item, sell1_format)
    elif int(item) == week_num+1:
        worksheet.write(row, col, item, sell2_format)
    elif int(item) == week_num-1:
        worksheet.write(row, col, item, sell3_format)
    else:
        worksheet.write(row, col, item, bold_format)
    worksheet.write(row, col + 1, cost)
    row += 1

for buy in ['B3:B14', 'B18:B22', 'B26:B29', 'G3:G33', 'L3:L50']:
    worksheet.conditional_format(buy, {'type': 'data_bar', 'bar_color': '#63C384'})
for sell in ['D3:D14', 'D18:D22', 'D26:D29', 'I3:I33', 'N3:N50']:
    worksheet.conditional_format(sell, {'type': 'data_bar', 'bar_color': '#FF9999'})


writer = pd.ExcelWriter(output_file, engine='openpyxl')
if os.path.exists(output_file):
    book = openpyxl.load_workbook(output_file)
    writer.book = book

df.to_excel(writer, sheet_name='Nifty', index=False)

writer.save()
writer.close()
print('Written')

C:\Users\vanandp\AppData\Local\Continuum\anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\vanandp\AppData\Local\Continuum\anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Written


In [58]:
dir(os.path)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_get_bothseps',
 '_getfinalpathname',
 '_getfullpathname',
 '_getvolumepathname',
 'abspath',
 'altsep',
 'basename',
 'commonpath',
 'commonprefix',
 'curdir',
 'defpath',
 'devnull',
 'dirname',
 'exists',
 'expanduser',
 'expandvars',
 'extsep',
 'genericpath',
 'getatime',
 'getctime',
 'getmtime',
 'getsize',
 'isabs',
 'isdir',
 'isfile',
 'islink',
 'ismount',
 'join',
 'lexists',
 'normcase',
 'normpath',
 'os',
 'pardir',
 'pathsep',
 'realpath',
 'relpath',
 'samefile',
 'sameopenfile',
 'samestat',
 'sep',
 'split',
 'splitdrive',
 'splitext',
 'splitunc',
 'stat',
 'supports_unicode_filenames',
 'sys']

In [59]:
import string
uc = string.ascii_uppercase
uc = list(uc)
cc = ['A', 'B']
resize_col  = dict()
for first in cc:
    for second in uc:
        resize_col[second] = 8
        resize_col[first+second] = 8

'{}'.format(resize_col)


"{'A': 8, 'AA': 8, 'B': 8, 'AB': 8, 'C': 8, 'AC': 8, 'D': 8, 'AD': 8, 'E': 8, 'AE': 8, 'F': 8, 'AF': 8, 'G': 8, 'AG': 8, 'H': 8, 'AH': 8, 'I': 8, 'AI': 8, 'J': 8, 'AJ': 8, 'K': 8, 'AK': 8, 'L': 8, 'AL': 8, 'M': 8, 'AM': 8, 'N': 8, 'AN': 8, 'O': 8, 'AO': 8, 'P': 8, 'AP': 8, 'Q': 8, 'AQ': 8, 'R': 8, 'AR': 8, 'S': 8, 'AS': 8, 'T': 8, 'AT': 8, 'U': 8, 'AU': 8, 'V': 8, 'AV': 8, 'W': 8, 'AW': 8, 'X': 8, 'AX': 8, 'Y': 8, 'AY': 8, 'Z': 8, 'AZ': 8, 'BA': 8, 'BB': 8, 'BC': 8, 'BD': 8, 'BE': 8, 'BF': 8, 'BG': 8, 'BH': 8, 'BI': 8, 'BJ': 8, 'BK': 8, 'BL': 8, 'BM': 8, 'BN': 8, 'BO': 8, 'BP': 8, 'BQ': 8, 'BR': 8, 'BS': 8, 'BT': 8, 'BU': 8, 'BV': 8, 'BW': 8, 'BX': 8, 'BY': 8, 'BZ': 8}"

In [60]:
df.head()

,Series,Industry,Company Name,Symbol,52wHigh,52wLow,52wDiff,52wMid,avgPrice,avgP-52wMid,open,lastPrice,prevClose,closePrice,basePrice,dayHigh,dayLow,dayDiff,change,pChange,52wHighDate,52wLowDate,52wDateDiff,ALGO_SCORE,ISIN Code,TAGS,TAG_SCORE,adhocMargin,applicableMargin,bcEndDate,bcStartDate,buyPrice1,buyPrice2,buyPrice3,buyPrice4,buyPrice5,buyQuantity1,buyQuantity2,buyQuantity3,buyQuantity4,buyQuantity5,cm_ffm,companyName,css_status_desc,deliveryQuantity,deliveryToTradedQuantity,exDate,extremeLossMargin,faceValue,indexVar,isExDateFlag,isinCode,marketType,ndEndDate,ndStartDate,priceBand,pricebandlower,pricebandupper,purpose,quantityTraded,recordDate,secDate,securityVar,sellPrice1,sellPrice2,sellPrice3,sellPrice4,sellPrice5,sellQuantity1,sellQuantity2,sellQuantity3,sellQuantity4,sellQuantity5,series,surv_indicator,symbol,totalBuyQuantity,totalSellQuantity,totalTradedValue,totalTradedVolume,varMargin
0,EQ,ENERGY,Jaiprakash Power Ventures Ltd.,JPPOWER,10.20,2.40,7.8,6.3,2.94,-3.36,2.85,2.95,2.85,2.90,2.85,3.00,2.85,0.15,0.10,3.51,2018-01-01,2018-07-24,204 days,5.5,INE351F01018,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,25.31,15-SEP-17,08-SEP-17,2.90,-,-,-,-,123295,-,-,-,-,1095.47,Jaiprakash Power Ventures Limited,Listed,1242475,36.98,06-SEP-17,6.06,10,-,0,INE351F01018,N,-,-,20,2.30,3.40,ANNUAL GENERAL MEETING,3359651,-,17AUG2018,19.25,-,-,-,-,-,-,-,-,-,-,EQ,-,JPPOWER,123295,-,98.77,3359651,19.25
1,EQ,CONSTRUCTION,Unitech Ltd.,UNITECH,12.25,3.95,8.3,8.1,4.25,-3.85,4.10,4.25,4.05,4.30,4.05,4.35,4.10,0.25,0.20,4.94,2018-01-02,2018-07-17,196 days,5.5,INE694A01020,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,15.20,28-SEP-17,23-SEP-17,4.30,-,-,-,-,5807620,-,-,-,-,956.26,Unitech Limited,Listed,9031033,43.08,21-SEP-17,5.00,2,-,0,INE694A01020,N,-,-,20,3.25,4.85,ANNUAL GENERAL MEETING,20965259,-,17AUG2018,10.20,-,-,-,-,-,-,-,-,-,-,EQ,-,UNITECH,5807620,-,891.02,20965259,10.20
2,EQ,ENERGY,RattanIndia Power Ltd.,RTNPOWER,9.50,3.10,6.4,6.3,4.14,-2.16,4.10,4.10,4.05,4.10,4.05,4.25,4.10,0.15,0.05,1.23,2017-11-21,2018-07-06,227 days,5.5,INE399K01017,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,17.44,19-SEP-17,13-SEP-17,-,-,-,-,-,-,-,-,-,-,556.92,RattanIndia Power Limited,Listed,995035,56.54,11-SEP-17,5.00,10,-,0,INE399K01017,N,-,-,20,3.25,4.85,ANNUAL GENERAL MEETING,1759737,-,17AUG2018,12.44,-,-,-,-,-,-,-,-,-,-,EQ,-,RTNPOWER,-,-,72.85,1759737,12.44
3,EQ,CONSUMER GOODS,Bajaj Hindusthan Sugar Ltd.,BAJAJHIND,18.80,5.60,13.2,12.2,6.56,-5.64,6.55,6.50,6.50,6.55,6.50,6.65,6.50,0.15,0.00,0.00,2018-01-02,2018-07-20,199 days,5.5,INE306A01021,"nifty500, niftymidsmallcap400, niftysmallcap250",3,-,14.81,15-SEP-17,09-SEP-17,-,-,-,-,-,-,-,-,-,-,601.41,Bajaj Hindusthan Sugar Limited,Listed,404411,60.69,07-SEP-17,5.00,1,-,0,INE306A01021,N,-,-,20,5.20,7.80,ANNUAL GENERAL MEETING,666333,-,17AUG2018,9.81,6.55,-,-,-,-,35586,-,-,-,-,EQ,-,BAJAJHIND,-,35586,43.71,666333,9.81
4,EQ,INDUSTRIAL MANUFACTURING,Suzlon Energy Ltd.,SUZLON,17.75,6.85,10.9,12.3,7.24,-5.06,7.25,7.20,7.20,7.25,7.20,7.30,7.20,0.10,0.00,0.00,2017-08-16,2018-07-19,337 days,12.5,INE040H01021,"nifty200, nifty500, niftylargemidcap250, nifty...",5,-,17.09,27-JUL-18,21-JUL-18,-,-,-,-,-,-,-,-,-,-,2391.24,Suzlon Energy Limited,Listed,4274193,38.17,19-JUL-18,5.00,2,-,0,INE040H01021,N,-,-,No Band,6.50,7.90,ANNUAL GENERAL MEETING,11198311,-,17AUG2018,12.09,7.25,-,-,-,-,259416,-,-,-,-,EQ,-,SUZLON,-,259416,810.76,11198311,12.09


In [61]:
list(month_buy.items())

[('August', 151),
 ('July', 147),
 ('June', 56),
 ('September', 53),
 ('May', 30),
 ('March', 17),
 ('February', 17),
 ('October', 14),
 ('November', 8),
 ('April', 3),
 ('December', 3),
 ('January', 1)]